In [1]:
#basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#libraries for text processing
import string
import re
import emoji
import nltk.stem
import nltk.chunk

#this is a language detection library, I used it for filter out non-English comments, but its accuracy needs further investigation
import langid

#function libraries
import load_data_utils
import parse_comments_utils
import features_utils
import classifier_utils
import plot_utils
from sklearn.model_selection import train_test_split

#libraries for connect to database
import datetime
from sqlalchemy import (create_engine, Column, Integer, String, Float, Text,
                        BigInteger, DateTime, ForeignKey, Boolean)
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
import config
import psycopg2

/anaconda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
# Database Configureation Constants
HOST = config.DATABASE_HOST
USERNAME = config.DATABASE_USERNAME
PASSWORD = config.DATABASE_PASSWORD
DBNAME = config.DATABASE_NAME
PORT = config.DATABASE_PORT

# Configuring DB Connection
connection_sting = "postgresql://{}:{}@{}:{}/{}".format(USERNAME,PASSWORD,HOST,PORT,DBNAME)
engine = create_engine(connection_sting)
#Base = declarative_base()

In [3]:
# Load and Parse Comments
# use functions in load_data_util.py and parse_comments_utils.py
data_200k,data,parsed_comments_200k,parsed_comments = load_data_utils.load_process_comments(connection_sting)
print(parsed_comments.shape)
parsed_comments

2017-08-22 21:15:55,729 : INFO : initializing identifier


(33023, 11)


,original_comment,full_text_ngramT,text_onlyT,emojiT,translated_emojiT,usernameT,hashtagT,punctuation,language,label,purchase_intent
0,Would mean a lot if you guys liked my last pos...,"[would, mean, a, lot, if, you, guy, like, my, ...","[would, mean, a, lot, if, you, guy, like, my, ...","[😄, 🔥, ❤]","[smiling_face_with_open_mouth_&_smiling_eyes, ...",[],[],,en,-2,0
1,This is STUNNING,"[this, is, stun]","[this, is, stun]",[],[],[],[],,en,1,0
2,❤️,[red_heart],[],[❤],[red_heart],[],[],,en,1,0
3,👌,[ok_hand],[],[👌],[ok_hand],[],[],,en,1,0
4,This is fire 🔥,"[this, is, fire, fire]","[this, is, fire]",[🔥],[fire],[],[],,en,1,0
5,@gulsanmannan the gloss 😍😍,"[the, gloss, smiling_face_with_heart-eyes, smi...","[the, gloss]","[😍, 😍]","[smiling_face_with_heart-eyes, smiling_face_wi...",[gulsanmannan],[],,en,2,0
6,😍😍,"[smiling_face_with_heart-eyes, smiling_face_wi...",[],"[😍, 😍]","[smiling_face_with_heart-eyes, smiling_face_wi...",[],[],,en,3,1
7,💋📷🔛🔝,"[kiss_mark, camera, on!_arrow, top_arrow]",[],"[💋, 📷, 🔛, 🔝]","[kiss_mark, camera, on!_arrow, top_arrow]",[],[],,en,3,1
8,Would mean a lot if you guys liked my last pos...,"[would, mean, a, lot, if, you, guy, like, my, ...","[would, mean, a, lot, if, you, guy, like, my, ...","[😄, 🔥, ❤]","[smiling_face_with_open_mouth_&_smiling_eyes, ...",[],[],,en,-2,0
9,Amazing,[],[],[],[],[],[],,it,1,0


In [10]:
parsed_comments_200k.to_pickle('./parsed_comments_200k')
parsed_comments.to_pickle('./parsed_comments')

In [2]:
parsed_comments_200k = pd.read_pickle('./parsed_comments_200k')
parsed_comments = pd.read_pickle('./parsed_comments')

In [11]:
parsed_comments_200k

,original_comment,full_text_ngramT,text_onlyT,emojiT,translated_emojiT,usernameT,hashtagT,punctuation,language,label,purchase_intent
0,Would mean a lot if you guys liked my last pos...,"[would, mean, a, lot, if, you, guy, like, my, ...","[would, mean, a, lot, if, you, guy, like, my, ...","[😄, 🔥, ❤]","[smiling_face_with_open_mouth_&_smiling_eyes, ...",[],[],,en,-2.0,0.0
1,This is STUNNING,"[this, is, stun]","[this, is, stun]",[],[],[],[],,en,1.0,0.0
2,❤️,[red_heart],[],[❤],[red_heart],[],[],,en,1.0,0.0
3,👌,[ok_hand],[],[👌],[ok_hand],[],[],,en,1.0,0.0
4,This is fire 🔥,"[this, is, fire, fire]","[this, is, fire]",[🔥],[fire],[],[],,en,1.0,0.0
5,@gulsanmannan the gloss 😍😍,"[the, gloss, smiling_face_with_heart-eyes, smi...","[the, gloss]","[😍, 😍]","[smiling_face_with_heart-eyes, smiling_face_wi...",[gulsanmannan],[],,en,2.0,0.0
6,😍😍,"[smiling_face_with_heart-eyes, smiling_face_wi...",[],"[😍, 😍]","[smiling_face_with_heart-eyes, smiling_face_wi...",[],[],,en,3.0,1.0
7,💋📷🔛🔝,"[kiss_mark, camera, on!_arrow, top_arrow]",[],"[💋, 📷, 🔛, 🔝]","[kiss_mark, camera, on!_arrow, top_arrow]",[],[],,en,3.0,1.0
8,Would mean a lot if you guys liked my last pos...,"[would, mean, a, lot, if, you, guy, like, my, ...","[would, mean, a, lot, if, you, guy, like, my, ...","[😄, 🔥, ❤]","[smiling_face_with_open_mouth_&_smiling_eyes, ...",[],[],,en,-2.0,0.0
9,Amazing,[],[],[],[],[],[],,it,1.0,0.0


In [12]:
features_utils.sentiment_analysis(parsed_comments)
features_utils.categorical_features(parsed_comments)
parsed_comments

,original_comment,full_text_ngramT,text_onlyT,emojiT,translated_emojiT,usernameT,hashtagT,punctuation,language,label,purchase_intent,compound,pos,neu,neg,has_exag,has_ques,has_at,num_emoji
0,Would mean a lot if you guys liked my last pos...,"[would, mean, a, lot, if, you, guy, like, my, ...","[would, mean, a, lot, if, you, guy, like, my, ...","[😄, 🔥, ❤]","[smiling_face_with_open_mouth_&_smiling_eyes, ...",[],[],,en,-2,0,0.7269,0.316,0.595,0.089,0,0,0,3
1,This is STUNNING,"[this, is, stun]","[this, is, stun]",[],[],[],[],,en,1,0,0.0000,0.000,1.000,0.000,0,0,0,0
2,❤️,[red_heart],[],[❤],[red_heart],[],[],,en,1,0,0.0000,0.000,1.000,0.000,0,0,0,1
3,👌,[ok_hand],[],[👌],[ok_hand],[],[],,en,1,0,0.6597,1.000,0.000,0.000,0,0,0,1
4,This is fire 🔥,"[this, is, fire, fire]","[this, is, fire]",[🔥],[fire],[],[],,en,1,0,-0.5859,0.000,0.294,0.706,0,0,0,1
5,@gulsanmannan the gloss 😍😍,"[the, gloss, smiling_face_with_heart-eyes, smi...","[the, gloss]","[😍, 😍]","[smiling_face_with_heart-eyes, smiling_face_wi...",[gulsanmannan],[],,en,2,0,0.7184,0.429,0.571,0.000,0,0,1,2
6,😍😍,"[smiling_face_with_heart-eyes, smiling_face_wi...",[],"[😍, 😍]","[smiling_face_with_heart-eyes, smiling_face_wi...",[],[],,en,3,1,0.7184,0.500,0.500,0.000,0,0,0,2
7,💋📷🔛🔝,"[kiss_mark, camera, on!_arrow, top_arrow]",[],"[💋, 📷, 🔛, 🔝]","[kiss_mark, camera, on!_arrow, top_arrow]",[],[],,en,3,1,0.5983,0.495,0.505,0.000,0,0,0,4
8,Would mean a lot if you guys liked my last pos...,"[would, mean, a, lot, if, you, guy, like, my, ...","[would, mean, a, lot, if, you, guy, like, my, ...","[😄, 🔥, ❤]","[smiling_face_with_open_mouth_&_smiling_eyes, ...",[],[],,en,-2,0,0.7269,0.316,0.595,0.089,0,0,0,3
9,Amazing,[],[],[],[],[],[],,it,1,0,0.0000,0.000,0.000,0.000,0,0,0,0


In [13]:
dim = 50
parsed_comments = features_utils.text_vectorization(parsed_comments_200k,parsed_comments,dim)
parsed_comments

2017-08-23 19:33:35,203 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-08-23 19:33:35,363 : INFO : adding document #10000 to Dictionary(3082 unique tokens: ['would', 'mean', 'lot', 'guy', 'like']...)
2017-08-23 19:33:35,506 : INFO : adding document #20000 to Dictionary(4189 unique tokens: ['would', 'mean', 'lot', 'guy', 'like']...)
2017-08-23 19:33:35,647 : INFO : adding document #30000 to Dictionary(5015 unique tokens: ['would', 'mean', 'lot', 'guy', 'like']...)
2017-08-23 19:33:35,809 : INFO : adding document #40000 to Dictionary(5723 unique tokens: ['would', 'mean', 'lot', 'guy', 'like']...)
2017-08-23 19:33:35,959 : INFO : adding document #50000 to Dictionary(6308 unique tokens: ['would', 'mean', 'lot', 'guy', 'like']...)
2017-08-23 19:33:36,119 : INFO : adding document #60000 to Dictionary(6867 unique tokens: ['would', 'mean', 'lot', 'guy', 'like']...)
2017-08-23 19:33:36,268 : INFO : adding document #70000 to Dictionary(7251 unique tokens: ['would', 'mean', '

2017-08-23 19:33:44,544 : INFO : topic #3(25.986): 0.988*"love" + -0.103*"red_heart" + 0.048*"two_hearts" + 0.045*"color" + -0.032*"smiling_face_with_heart-eyes" + 0.023*"palett" + 0.022*"omg" + 0.020*"purple_heart" + 0.019*"much" + 0.018*"sparkling_heart"
2017-08-23 19:33:44,545 : INFO : topic #4(20.222): 0.678*"beauti" + 0.663*"two_hearts" + 0.227*"need" + 0.123*"omg" + -0.060*"love" + 0.055*"sparkling_heart" + 0.052*"koko" + 0.048*"color" + 0.048*"smiling_cat_face_with_heart-eyes" + 0.045*"sparkles"
2017-08-23 19:33:44,848 : INFO : preparing a new chunk of documents
2017-08-23 19:33:44,965 : INFO : using 100 extra samples and 2 power iterations
2017-08-23 19:33:44,966 : INFO : 1st phase: constructing (10249, 150) action matrix
2017-08-23 19:33:45,115 : INFO : orthonormalizing (10249, 150) action matrix
2017-08-23 19:33:45,484 : INFO : 2nd phase: running dense svd on (150, 20000) matrix
2017-08-23 19:33:46,281 : INFO : computing the final decomposition
2017-08-23 19:33:46,282 : INFO 

2017-08-23 19:33:51,796 : INFO : topic #3(40.984): 0.981*"love" + -0.096*"red_heart" + 0.062*"makeup" + 0.048*"two_hearts" + 0.045*"look" + 0.041*"omg" + 0.036*"much" + -0.036*"smiling_face_with_heart-eyes" + -0.034*"unicorn_face" + 0.031*"tri"
2017-08-23 19:33:51,797 : INFO : topic #4(32.846): 0.757*"beauti" + 0.532*"makeup" + 0.246*"two_hearts" + 0.119*"omg" + -0.103*"love" + 0.103*"look" + 0.076*"lipstick" + 0.051*"like" + 0.048*"purple_heart" + 0.047*"without"
2017-08-23 19:33:52,107 : INFO : preparing a new chunk of documents
2017-08-23 19:33:52,225 : INFO : using 100 extra samples and 2 power iterations
2017-08-23 19:33:52,226 : INFO : 1st phase: constructing (10249, 150) action matrix
2017-08-23 19:33:52,383 : INFO : orthonormalizing (10249, 150) action matrix
2017-08-23 19:33:52,873 : INFO : 2nd phase: running dense svd on (150, 20000) matrix
2017-08-23 19:33:53,585 : INFO : computing the final decomposition
2017-08-23 19:33:53,586 : INFO : keeping 50 factors (discarding 16.261

2017-08-23 19:33:59,442 : INFO : topic #2(74.725): 0.997*"yellow_heart" + 0.063*"red_heart" + 0.028*"face_with_tears_of_joy" + -0.025*"unicorn_face" + -0.011*"smiling_face_with_heart-eyes" + 0.011*"love" + 0.010*"blue_heart" + 0.006*"sparkles" + 0.006*"purple_heart" + 0.006*"fire"
2017-08-23 19:33:59,444 : INFO : topic #3(72.024): 0.983*"red_heart" + -0.109*"unicorn_face" + 0.094*"love" + -0.074*"smiling_face_with_heart-eyes" + -0.067*"yellow_heart" + 0.020*"people_with_bunny_ears_partying" + 0.018*"beauti" + 0.017*"fire" + 0.012*"face_with_tears_of_joy" + 0.011*"face_blowing_a_kiss"
2017-08-23 19:33:59,445 : INFO : topic #4(55.349): 0.999*"people_with_bunny_ears_partying" + -0.023*"red_heart" + -0.022*"unicorn_face" + 0.018*"two_hearts" + 0.012*"male_sign" + 0.007*"female_sign" + 0.006*"sparkling_heart" + -0.006*"love" + 0.005*"sparkles" + 0.004*"face_blowing_a_kiss"
2017-08-23 19:33:59,680 : INFO : preparing a new chunk of documents
2017-08-23 19:33:59,767 : INFO : using 100 extra sa

,original_comment,full_text_ngramT,text_onlyT,emojiT,translated_emojiT,usernameT,hashtagT,punctuation,language,label,...,tfidf_45,tfidf_46,tfidf_47,tfidf_48,tfidf_49,tfidf_5,tfidf_6,tfidf_7,tfidf_8,tfidf_9
0,Would mean a lot if you guys liked my last pos...,"[would, mean, a, lot, if, you, guy, like, my, ...","[would, mean, a, lot, if, you, guy, like, my, ...","[😄, 🔥, ❤]","[smiling_face_with_open_mouth_&_smiling_eyes, ...",[],[],,en,-2,...,-0.044127,-0.001127,-0.010525,0.002418,0.000977,-0.000704,0.013542,0.003695,0.010201,0.017213
1,This is STUNNING,"[this, is, stun]","[this, is, stun]",[],[],[],[],,en,1,...,0.004509,-0.001191,-0.002825,0.000246,-0.001714,0.001988,0.004028,0.001194,0.000492,0.003753
2,❤️,[red_heart],[],[❤],[red_heart],[],[],,en,1,...,-0.001123,-0.001656,0.000348,0.002388,0.000312,-0.101521,-0.020932,-0.016238,-0.012789,-0.010250
3,👌,[ok_hand],[],[👌],[ok_hand],[],[],,en,1,...,-0.009676,-0.012143,0.010457,0.003199,-0.004327,0.006842,0.022862,0.002365,0.017356,0.020017
4,This is fire 🔥,"[this, is, fire, fire]","[this, is, fire]",[🔥],[fire],[],[],,en,1,...,0.002103,0.002002,-0.000541,0.003849,0.000311,0.003886,0.033214,0.004546,0.017055,0.048087
5,@gulsanmannan the gloss 😍😍,"[the, gloss, smiling_face_with_heart-eyes, smi...","[the, gloss]","[😍, 😍]","[smiling_face_with_heart-eyes, smiling_face_wi...",[gulsanmannan],[],,en,2,...,-0.000273,-0.000936,0.000979,0.001461,0.000659,-0.017971,-0.011855,-0.010064,-0.002640,-0.003936
6,😍😍,"[smiling_face_with_heart-eyes, smiling_face_wi...",[],"[😍, 😍]","[smiling_face_with_heart-eyes, smiling_face_wi...",[],[],,en,3,...,-0.001210,0.002710,-0.000484,0.002559,0.000160,-0.037496,-0.024181,-0.020208,-0.005408,-0.008433
7,💋📷🔛🔝,"[kiss_mark, camera, on!_arrow, top_arrow]",[],"[💋, 📷, 🔛, 🔝]","[kiss_mark, camera, on!_arrow, top_arrow]",[],[],,en,3,...,-0.009758,0.006156,0.000274,0.003023,-0.001294,0.002965,0.005375,0.001816,0.001190,0.006120
8,Would mean a lot if you guys liked my last pos...,"[would, mean, a, lot, if, you, guy, like, my, ...","[would, mean, a, lot, if, you, guy, like, my, ...","[😄, 🔥, ❤]","[smiling_face_with_open_mouth_&_smiling_eyes, ...",[],[],,en,-2,...,-0.044127,-0.001127,-0.010525,0.002418,0.000977,-0.000704,0.013542,0.003695,0.010201,0.017213
9,Amazing,[],[],[],[],[],[],,it,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
parsed_comments.to_pickle('./parsed_comments_allfeature')

In [2]:
parsed_comments = pd.read_pickle('./parsed_comments_allfeature')

In [3]:
parsed_comments = features_utils.clean_features(parsed_comments)

/Users/mengyuan/Documents/Insight/InsightCherryPick/Processing/features_utils.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  features['num_emoji'] = features['num_emoji'].copy()/100.


In [5]:
parsed_comments.iloc[0]
parsed_comments.shape

(28846, 69)

In [6]:
features = parsed_comments.loc[0:,'compound':]
labels = parsed_comments.loc[0:,'label':'purchase_intent']
print(features)
print(labels)
print(features.shape)

        compound    pos    neu    neg  has_exag  has_ques  has_at  num_emoji  \
0         0.7269  0.316  0.595  0.089         0         0       0       0.03   
1         0.0000  0.000  1.000  0.000         0         0       0       0.00   
2         0.0000  0.000  1.000  0.000         0         0       0       0.01   
3         0.6597  1.000  0.000  0.000         0         0       0       0.01   
4        -0.5859  0.000  0.294  0.706         0         0       0       0.01   
5         0.7184  0.429  0.571  0.000         0         0       1       0.02   
6         0.7184  0.500  0.500  0.000         0         0       0       0.02   
7         0.5983  0.495  0.505  0.000         0         0       0       0.04   
8         0.7269  0.316  0.595  0.089         0         0       0       0.03   
10        0.6597  0.643  0.357  0.000         0         0       0       0.02   
11        0.8442  1.000  0.000  0.000         0         0       0       0.01   
12       -0.2500  0.000  0.818  0.182   

In [7]:
features.to_pickle('./features')
labels.to_pickle('./labels')

In [2]:
features = pd.read_pickle('./features')
labels = pd.read_pickle('./labels')

# go to notebook 02